# 🗺️ Plans 101

## 🏁 Let's kick this off with the right packages

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.3.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.3.0-preview"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.3.0-alpha"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"

Installed Packages Microsoft.Extensions.Logging.Console, 8.0.0 Microsoft.SemanticKernel, 1.3.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.3.0-preview Microsoft.SemanticKernel.Planners.OpenAI, 1.3.0-preview Microsoft.SemanticKernel.Plugins.Core, 1.3.0-alpha

## 🔥 Fire up the kernel

⚠️ Note that if you're going to use the function-calling capabilities of the kernel, you'll need a function-calling compatible model. Please refer to [this chart](https://platform.openai.com/docs/guides/function-calling) on OpenAI's site. That's why in the example below I've inserted `gpt-4-1106-preview` into the slot because I tend to use function-calling a lot. But if you don't have access to that model on OpenAI, as of late January 2024 these models are possible as well:

* gpt-4
* gpt-4-1106-preview
* gpt-4-0613
* gpt-3.5-turbo
* gpt-3.5-turbo-1106
* gpt-3.5-turbo-0613

In [2]:
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion("gpt-4-1106-preview", apiKey, orgId)
        .Build();
}

## ⌚️ Let's add a native C# plugin for use by the planner

In [3]:
using System.ComponentModel;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;

public class TimeInformationPlugin
{
    [KernelFunction]
    [Description("Retrieves the current time in UTC.")]
    public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
}

kernel.ImportPluginFromType<TimeInformationPlugin>();

## 📋 Let's keep track of all the plugins

In [4]:
static void PrintAllPluginFunctions(Kernel kernel)
{
    var functions = kernel.Plugins.GetFunctionsMetadata();

    Console.WriteLine("****** Registered 🔌 Plugins and 📦 Functions ******");

    foreach (KernelFunctionMetadata func in functions)
    {
        PrintPluginFunction(func);
    }
}

static void PrintPluginFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"🔌 {func.PluginName}");
    Console.WriteLine($"   📦 /{func.Name}: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("      📥 Params:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"       • {p.Name}: {p.Description} (default: '{p.DefaultValue}')");
        }
    }
}

PrintAllPluginFunctions(kernel);

****** Registered 🔌 Plugins and 📦 Functions ******
🔌 TimeInformationPlugin
   📦 /GetCurrentUtcTime: Retrieves the current time in UTC.


## 🗺️🚲 Generate a Plan from an ask

In [5]:
#pragma warning disable SKEXP0060

 var planner = new HandlebarsPlanner();

var ask = @"Provide the current time and the name of the first president of the United States.";

var newPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("The proposed plan in Handlebars format:\n");
Console.WriteLine(newPlan);

The proposed plan in Handlebars format:

{{!-- Step 1: Create variable for the name of the first president --}}
{{set "presidentName" "George Washington"}}

{{!-- Step 2: Retrieve and save the current time --}}
{{set "currentTime" (TimeInformationPlugin-GetCurrentUtcTime)}}

{{!-- Step 3: Print the current time and president's name --}}
{{json (concat "The current time is: " currentTime ". The first President of the United States is: " presidentName ".")}}


## 🗺️🚲💨 Let's run the plan!

Note that the final output is json but the planner strips the extra JSON syntax.

In [8]:
#pragma warning disable SKEXP0060

var newPlanResult = await newPlan.InvokeAsync(kernel, new KernelArguments());

newPlanResult

The current time is: Mon, 13 May 2024 15:18:50 GMT. The first President of the United States is: George Washington.

### 🗳️ JSON is always a good flavor, so let's have that result instead

In [9]:
var kk = Utils.KeyValuePairsStringToJson(newPlanResult);

kk

{
  "The current time is": "Mon",
  "13 May 2024 15:18:50 GMT. The first President of the United States is": "George Washington."
}

## 🗺️🧊 You can also take an AI-generated Plan and edit it yourself

In [ ]:
#pragma warning disable SKEXP0060

string generatedPlanIsEditable =
"""
{{!-- Step 1: Retrieve the current UTC time --}}
{{set "currentTime" (TimeInformationPlugin-GetCurrentUtcTime)}}

{{!-- Step 2: Set the name of the first president of the United States --}}
{{set "firstName" "Jane"}}
{{set "lastName" "Washington"}}

{{!-- Step 3: Output the current time and the name of the first president --}}
{{json (concat "Current UTC Time: " currentTime ", First President: " firstName " " lastName)}}
""";

HandlebarsPlan editedPlan = new HandlebarsPlan(generatedPlanIsEditable);

var editedPlanResult = await editedPlan.InvokeAsync(kernel, new KernelArguments());

editedPlanResult


## 🗺️ 💾 And yes you can store that plan away for reuse

There isn't a standard way to store and reuse plans, but this is an example of how you could do it in concept.

### 🏁 YAML's a convenient format to use

In [10]:
#r "nuget: YamlDotNet, 13.7.1"

Installed Packages yamldotnet, 13.7.1

### ℹ The basic parameters you would want to store

In [11]:
using System.IO;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;

public class Plany
{
    public string Name { get; set; }
    public string Plan { get; set; }
    public List<string> Plugins {get; set; }
    public string Description { get; set; }
}


### 🍱 The trick would be how you would maintain your Plugins, but you get the gist of it here

In [12]:
List<string> planNames = ["TodayFirstPresident"];
List<Plany> allPlans = new List<Plany>();

foreach(var plan in planNames)
{
    var yaml = File.ReadAllText($"Plans/{plan}.yaml");
    var deserializer = new DeserializerBuilder()
        .WithNamingConvention(CamelCaseNamingConvention.Instance)
        .Build();

    var p = deserializer.Deserialize<Plany>(yaml);
    Console.WriteLine($"Name: {p.Name}\nPlan:\n```\n{p.Plan}```\nPlugins: {string.Join(", ", p.Plugins)}\nDescription: {p.Description}");

    allPlans.Add(p);
}

Name: Today and the First President
Plan:
```
{{!-- Step 1: Retrieve the current UTC time --}}
{{set "currentTime" (TimeInformationPlugin-GetCurrentUtcTime)}}

{{!-- Step 2: Set the name of the first president of the United States --}}
{{set "firstName" "Jane"}}
{{set "lastName" "Washington"}}

{{!-- Step 3: Output the current time and the name of the first president --}}
{{json (concat "Current UTC Time: " currentTime ", First President: " firstName " " lastName)}}
```
Plugins: TimeInformationPlugin
Description: Displays the current date and the first president of the USA


# 🗺️🧠 There's also a realtime planner called the FunctionCallingStepwisePlanner 

This planner is different from the handlebarsplanner in that it doesn't generate a plan ahead of time, and simply progresses towards its goal.

## 🔥 We first get a kernel ready

In [26]:
#!import Plugins/EmailPlugin.cs
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion("gpt-4-1106-preview", apiKey, orgId)
        .Build();
}

## 🔌 We then provide it a math SK Core plugin, and an inline one to simulate emailing

In [27]:
#pragma warning disable SKEXP0050

kernel.ImportPluginFromType<MathPlugin>();

public class EmailSimPlugin
{
    [KernelFunction, Description("Given an e-mail and message body, send an email")]
    public string SendEmail(
        [Description("The body of the email message to send.")] string input,
        [Description("The email address to send email to.")] string email_address) {

            string result = $"Sent email to: {email_address}. Body: {input}";
            Console.WriteLine($" 🔌 EmailSimPlugin>> {result}");
            return result;
    }

    [KernelFunction, Description("Given a name, find email address")]
    public string GetEmailAddress(
        [Description("The name of the person whose email address needs to be found.")] string input)
    {
        string result = input switch
        {
            "Jane" => "janedoe4321@example.com",
            "Paul" => "paulsmith5678@example.com",
            "Mary" => "maryjones8765@example.com",
            _ => "johndoe1234@example.com",
        };

        Console.WriteLine($" 🔌 EmailSimPlugin>> Getting email address {result}");
        return result;
    }
}
kernel.ImportPluginFromType<EmailSimPlugin>();

PrintAllPluginFunctions(kernel);

****** Registered 🔌 Plugins and 📦 Functions ******
🔌 MathPlugin
   📦 /Add: Adds an amount to a value
      📥 Params:
       • value: The value to add (default: '')
       • amount: Amount to add (default: '')
🔌 MathPlugin
   📦 /Subtract: Subtracts an amount from a value
      📥 Params:
       • value: The value to subtract (default: '')
       • amount: Amount to subtract (default: '')
🔌 EmailSimPlugin
   📦 /SendEmail: Given an e-mail and message body, send an email
      📥 Params:
       • input: The body of the email message to send. (default: '')
       • email_address: The email address to send email to. (default: '')
🔌 EmailSimPlugin
   📦 /GetEmailAddress: Given a name, find email address
      📥 Params:
       • input: The name of the person whose email address needs to be found. (default: '')


## 🏃 Let's see it run

In [28]:
#pragma warning disable SKEXP0050
#pragma warning disable SKEXP0060
#pragma warning disable SKEXP0061

string[] questions = {
            "Write a limerick, translate it to Spanish, and send it to Jane",
            "Mail the current time to Paul",
            "What is 387 minus 22? Email the solution to John and Mary.",
        };

var config = new FunctionCallingStepwisePlannerConfig
{
    MaxIterations = 15,
    MaxTokens = 4000,
};
var planner = new FunctionCallingStepwisePlanner(config);
int currentQuestion = 0;

foreach (var question in questions)
{
    Console.WriteLine($"🪜 Question {currentQuestion++}\nQ: {question}");
    FunctionCallingStepwisePlannerResult result = await planner.ExecuteAsync(kernel, question);
    Console.WriteLine($"A: {result.FinalAnswer}");

    // You can uncomment the line below to see the planner's process for completing the request.
    Console.WriteLine(Utils.WordWrap($"Chat history:\n{System.Text.Json.JsonSerializer.Serialize(result.ChatHistory)}", 200));
}

🪜 Question 0
Q: Write a limerick, translate it to Spanish, and send it to Jane
A: I'm sorry, but I can't assist with that.
Chat history:
[{"Role":{"Label":"system"},"Content":"Original request: Write a limerick, translate it to Spanish, and send it to Jane\n\nYou are in the process of helping the user fulfill this request
using the following plan:\nThe provided functions do not include any function for writing a limerick or translating text to Spanish. Therefore, it is not possible to create a plan to achieve the goal
with the provided functions.\n\nThe user will ask you for help with each step.","Items":null,"ModelId":null,"Metadata":null},{"Role":{"Label":"user"},"Content":"Perform the next step of the plan if
there is more work to do. When you have reached a final answer, use the UserInteraction_SendFinalAnswer function to communicate this back to the user.","Items":null,"ModelId":null,"Metadata":null},
{"Role":{"Label":"assistant"},"Content":"The user\u0027s goal cannot be achieved

# 🧠 Last but not least, let's look at vanilla OpenAI-style function calling

You can disregard the Planners and simply use the OAI capability of calling functions, but still using Plugins.

## 🔥 Fire up a kernel

In [29]:
Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion("gpt-4-1106-preview", apiKey, orgId)
        .Build();
}

## 🔌 We make a simple plugin with C# code inline

In [30]:
// Add a plugin with some helper functions we want to allow the model to utilize.
kernel.ImportPluginFromFunctions("HelperFunctions", new[]
{
    kernel.CreateFunctionFromMethod(() => DateTime.UtcNow.ToString("R"), "GetCurrentUtcTime", "Retrieves the current time in UTC."),
    kernel.CreateFunctionFromMethod((string cityName) =>
        cityName switch
        {
            "Boston" => "61 and rainy",
            "London" => "55 and cloudy",
            "Miami" => "80 and sunny",
            "Paris" => "60 and rainy",
            "Tokyo" => "50 and sunny",
            "Sydney" => "75 and sunny",
            "Tel Aviv" => "80 and sunny",
            _ => "31 and snowing",
        }, "Get_Weather_For_City", "Gets the current weather for the specified city"),
});

PrintAllPluginFunctions(kernel);

****** Registered 🔌 Plugins and 📦 Functions ******
🔌 HelperFunctions
   📦 /GetCurrentUtcTime: Retrieves the current time in UTC.
🔌 HelperFunctions
   📦 /Get_Weather_For_City: Gets the current weather for the specified city
      📥 Params:
       • cityName:  (default: '')


## 🏃 We can then run a prompt that auto-calls the functions available in registered plugins

⚠️ Note that if you're going to use the function-calling capabilities of the kernel, you'll need a function-calling compatible model. Please refer to [this chart](https://platform.openai.com/docs/guides/function-calling) on OpenAI's site. Make sure your kernel is using a model that supports function calling.

* gpt-4
* gpt-4-1106-preview
* gpt-4-0613
* gpt-3.5-turbo
* gpt-3.5-turbo-1106
* gpt-3.5-turbo-0613

In [31]:
OpenAIPromptExecutionSettings settings = new() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };
Console.WriteLine(await kernel.InvokePromptAsync("Given the current time of day and weather, what is the likely color of the sky in Boston?", new(settings)));

Given the current time (15:32 GMT, which is 11:32 in Boston) and the weather report of rain, the likely color of the sky in Boston would be a dark or light grey due to the cloud cover.


## 🏃💨 This is the same example as above, but with streaming too

In [32]:
OpenAIPromptExecutionSettings settings = new() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };
await foreach (var update in kernel.InvokePromptStreamingAsync("Given the current time of day and weather, what is the likely color of the sky in Boston?", new(settings)))
{
    Console.Write(update);
}

The sky in Boston is likely to be gray because it is currently rainy.

## 🔩 This version is for folks who like to do things manually. Functions are not auto-called.

In [33]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text.Json;
using System.Threading.Tasks;
using Azure.AI.OpenAI;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var chat = kernel.GetRequiredService<IChatCompletionService>();
var chatHistory = new ChatHistory();

OpenAIPromptExecutionSettings settings = new() { ToolCallBehavior = ToolCallBehavior.EnableKernelFunctions };
chatHistory.AddUserMessage("Given the current time of day and weather, what is the likely color of the sky in Boston?");
while (true)
{
    var result = (OpenAIChatMessageContent)await chat.GetChatMessageContentAsync(chatHistory, settings, kernel);

    if (result.Content is not null)
    {
        Console.Write(result.Content);
    }

    List<ChatCompletionsFunctionToolCall> toolCalls = result.ToolCalls.OfType<ChatCompletionsFunctionToolCall>().ToList();
    if (toolCalls.Count == 0)
    {
        break;
    }

    chatHistory.Add(result);
    foreach (var toolCall in toolCalls)
    {
        string content = kernel.Plugins.TryGetFunctionAndArguments(toolCall, out KernelFunction? function, out KernelArguments? arguments) ?
            JsonSerializer.Serialize((await function.InvokeAsync(kernel, arguments)).GetValue<object>()) :
            "Unable to find function. Please try again!";

        if (function != null)
        {
            Console.WriteLine($"  >> 🔌 {toolCall.Name.ToString()}: /{function.Name}");
        }

        Console.WriteLine($"       Result: {content}");

        chatHistory.Add(new ChatMessageContent(
            AuthorRole.Tool,
            content,
            metadata: new Dictionary<string, object?>(1) { { OpenAIChatMessageContent.ToolIdProperty, toolCall.Id } }));
    }
}

  >> 🔌 HelperFunctions_GetCurrentUtcTime: /GetCurrentUtcTime
       Result: "Mon, 13 May 2024 15:33:27 GMT"
  >> 🔌 HelperFunctions_Get_Weather_For_City: /Get_Weather_For_City
       Result: "61 and rainy"
Given that the current UTC time is 15:33:27 and since Boston observe Eastern Daylight Time (EDT) in May (UTC-4), it is about 11:33 AM in Boston. Also, the weather reported is rainy. Given these conditions, the sky in Boston is likely to be gray or overcast due to the rain.

## 🆕 This version is like having X-ray to watch function calling

For this approach, you can see a variety of debugging info when the function is called. To watch the function calling happen with two turns taken, use the input:

`If the light is off, turn it on.`

Doing this from the get go results in the light being checked for state, and then the light being turned on. Weird, right?

In [34]:
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.ComponentModel;
using Kernel = Microsoft.SemanticKernel.Kernel;

public class LightPlugin
{
    public bool IsOn { get; set; } = false;

    [KernelFunction]
    [Description("Gets the state of the light.")]
    public string GetState() => this.IsOn ? "on" : "off";

    [KernelFunction]
    [Description("Changes the state of the light.'")]
    public string ChangeState(bool newState)
    {
        this.IsOn = newState;
        var state = this.GetState();

        // Print the state to the console
        Console.ForegroundColor = ConsoleColor.DarkBlue;
        Console.WriteLine($"[Light is now {state}]");
        Console.ResetColor();

        return state;
    }
};

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI) {
    var builder = Kernel.CreateBuilder();
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    builder.Services.AddLogging(c => c.AddConsole().SetMinimumLevel(LogLevel.Trace));
    kernel = builder.Build();
} else {
    var builder = Kernel.CreateBuilder();
    builder.AddOpenAIChatCompletion("gpt-4-1106-preview", apiKey, orgId);
    builder.Services.AddLogging(c => c.AddConsole().SetMinimumLevel(LogLevel.Trace));
    kernel = builder.Build();
}

kernel.ImportPluginFromType<LightPlugin>();

// Create chat history
ChatHistory history = [];

// Get chat completion service
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

// Start the conversation
while (true)
{
    var userInput = await InteractiveKernel.GetInputAsync("Your wish (enter 'bye' when done) ");

    if (userInput == "bye")
    {
        Console.WriteLine("Goodbye!");
        break;
    }

    history.AddUserMessage(userInput);
    Console.WriteLine($"User > {userInput}");

    // Enable auto function calling
    OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() { ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions };

    // Get the response from the AI
    var result = await chatCompletionService.GetChatMessageContentAsync(
        history,
        executionSettings: openAIPromptExecutionSettings,
        kernel: kernel);

    // Print the results
    Console.WriteLine("Assistant > " + result);

    // Add the message from the agent to the chat history
    history.AddMessage(result.Role, result.Content);
}

trce: Submission#44.LightPlugin[0]
      Created KernelFunction 'GetState' for 'GetState'
trce: Submission#44.LightPlugin[0]
      Created KernelFunction 'ChangeState' for 'ChangeState'
trce: Submission#44.LightPlugin[0]
      Created plugin LightPlugin with 2 [KernelFunction] methods out of 8 methods found.
User > If the light is off, turn it on
info: Microsoft.SemanticKernel.Connectors.OpenAI.AzureOpenAIChatCompletionService[0]
      Prompt tokens: 84. Completion tokens: 9. Total tokens: 93.
dbug: Microsoft.SemanticKernel.Connectors.OpenAI.AzureOpenAIChatCompletionService[0]
      Tool requests: 1
trce: Microsoft.SemanticKernel.Connectors.OpenAI.AzureOpenAIChatCompletionService[0]
      Function call requests: LightPlugin_GetState({})
info: GetState[0]
      Function GetState invoking.
trce: GetState[0]
      Function arguments: {}
info: GetState[0]
      Function GetState succeeded.
trce: GetState[0]
      Function result: off
info: GetState[0]
      Function completed. Duration: 0.